## Generator and BESS Modelling

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy import interpolate
from scipy.optimize import minimize
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Inputs - Data Cleaning

In [ ]:
def coerce_power(power_obj):
    if isinstance(power_obj, str):
        value, unit = power_obj.split(" ", 2)
        match unit:
            case "kW":
                return float(value)
        match unit:
            case "W":
                return float(value) / 1000 # W -> kW
    else:
        return np.nan

In [ ]:
# load data
load_df = pd.read_csv("load-data.csv", parse_dates=True)

# parse datetimes
load_df["Time"] = pd.to_datetime(load_df["Time"], format="%d/%m/%Y %H:%M")

# coerce string power values into float kW values
load_df["Power"] = load_df["Power"].apply(lambda x: coerce_power(x))

load_df = load_df.set_index("Time")
load_df.head()

In [ ]:
load_df.mean()

**Assumption**: The data has mutiple values per timestamp. I have been told there is only one meter onsite and so assume these values or instanteous values so the best course of action is to average across them.

In [ ]:
total_load_df = load_df.groupby("Time").mean()
total_load_df.plot(figsize=(20,3), linestyle="--", marker=".", alpha=0.5)

In [ ]:
total_load_df = total_load_df.truncate(before="2025-05-13")
total_load_df.plot(figsize=(20,3), linestyle="--", marker=".", alpha=0.5)

**Assumption**: The data has a large gap as the start. Lets trim this and fill any remaining gaps with a basic ffill

In [ ]:
ts_start, ts_end = total_load_df.index[0], total_load_df.index[-1] 
full_spine_df = pd.DataFrame({"Time": pd.date_range(ts_start, ts_end, freq="1min")}).set_index("Time")
filled_load_df = full_spine_df.join(total_load_df).ffill().dropna()

In [ ]:
# filled_load_df.to_csv("../data/filled_load_df.csv")

## Question 1: Diesel Generator

Given no grid connection and load profile:

**Question**: What size of generator would you recommend?

In [ ]:
print(f"Max load = {filled_load_df["Power"].max()}kW")

In [ ]:
GEN_SIZE = 40 #kW rounding up to block of 10kW

**Answer**: Need to be able to meet peak demand using blocks of 10kW, so generator would have to be sized to 40kW.

**Question**: Calculate the total cost of power the site with the generator over the period covered in the load profile data?

**Answer**: Need to consider the rental price per day and the fuel price.

- Load, generator size and efficiency curve will give you litres of fuel. Cost of fuel is £1.43/l.
- Rental price depends only on the generator size @ £20/week/10kW block

To get a better idea of cost, we have imputed the gaps in the load profile using a forward filling method.

In [ ]:
PERIOD_WEEKS = (ts_end - ts_start) / dt.timedelta(days=7)
PERIOD_WEEKS

In [ ]:
GEN_PRICE_10KW_WEEK = 20 # £ per week
rental_price = (GEN_SIZE/10)*GEN_PRICE_10KW_WEEK*PERIOD_WEEKS
print(f"**RENTAL PRICE**: £{rental_price:.1f} over the period ({PERIOD_WEEKS:.1f} weeks) from {ts_start} to {ts_end}")

In [ ]:
gen_curve  = pd.read_csv("gen_fuel_eff_curves.csv", index_col=0)

In [ ]:
px.line(gen_curve)

#### **NOTE**: This generator seems very inefficient. 0.02 kWh/litre is odd, typical generators are more like 3-4 kWh/litre at full load. I think the given % based curve was out by a factor of 100. Manually corrected it.

#### Interpolate the fuel consumption for the load timeseries

In [ ]:
fuel_func = interpolate.interp1d(GEN_SIZE*gen_curve.index, gen_curve['kWh_litre'])

In [ ]:
filled_load_df["fuel_kWh_litre"] = [fuel_func(i) for i in filled_load_df["Power"]]

#### Calculate the energy kWh for each timestep and divide by fuel consumption to get litres of fuel used

In [ ]:
filled_load_df["Energy_kWh"] = filled_load_df["Power"]/60

In [ ]:
filled_load_df["Fuel_consump_litres"] = filled_load_df["Energy_kWh"]/filled_load_df["fuel_kWh_litre"]

#### Calculate cost based on 1.43£/l of fuel

In [ ]:
PRICE_PER_LITRE = 1.43

filled_load_df["Fuel_cost_£"] = filled_load_df["Fuel_consump_litres"] * PRICE_PER_LITRE

In [ ]:
(filled_load_df["Fuel_consump_litres"].sum())

In [ ]:
fuel_cost = filled_load_df["Fuel_cost_£"].sum()
print(f"**FUEL cost**: £{fuel_cost:.1f} over the period ({PERIOD_WEEKS:.1f} weeks) from {ts_start} to {ts_end}")

In [ ]:
total_gen_cost = fuel_cost + rental_price
total_gen_cost

In [ ]:
TOTAL_GEN_COST = 4601.59

In [ ]:
TOTAL_FUEL = filled_load_df["Fuel_consump_litres"].sum()
TOTAL_FUEL

**Question**: Calculate the Co2 emissions given 2.86kg/l

In [ ]:
CO2_LITRE = 2.86#kg

total_gen_co2 = filled_load_df["Fuel_consump_litres"].sum()*CO2_LITRE

f"The total co2 = {total_gen_co2:.1f}kg"

In [ ]:
filled_load_df["Fuel_consump_litres"].sum()*CO2_LITRE

In [ ]:
TOTAL_GEN_CO2 = 8426.1

## Question 2: Diesel generator coupled with a battery
The battery is able to charge from the generator and discharge to power the load.
We believe adding a battery would add a significant Co2 and £ savings to our customers but we need to quantify those savings and choose the best battery size


#### **Question**: What is the battery size (kW) and energy (kWh) that would minimise the generator size?

In [ ]:
#need for power/energy calculations
delta_t = pd.to_datetime(filled_load_df.index).diff().mean() / pd.Timedelta("1hr")
delta_t

In [ ]:
filled_load_df.isna().sum()

## Simulation
This model assumes the battery acting is as 'peak shaving'
Round trip eff of 85%, assuming, charging and discharging eff of sqrt(85%)=92%

In [ ]:
def simulate_battery_generator(
    load_ser: pd.Series,
    battery_capacity_kwh: float,
    battery_power_kw: float,
    generator_power_kw: float,
    normalised_gen_curve: pd.DataFrame,
    battery_efficiency: float = 0.92,
    initial_soc: float = 0.5,
) -> pd.DataFrame:
    battery_efficiency = 0.92
    time_step_hours = pd.to_datetime(load_ser.index).diff().mean() / pd.Timedelta("1hr")
    soc_kwh = initial_soc * battery_capacity_kwh
    max_charge_kwh = max_discharge_kwh = (battery_power_kw * time_step_hours)*battery_efficiency

    # efficiency curve scales to gen power
    fuel_func = interpolate.interp1d(
        generator_power_kw * normalised_gen_curve.index,
        normalised_gen_curve["kWh_litre"],
    )

    net_power_history = []
    soc_history = []
    fault_history = []
    battery_power_history = []
    gen_power_history = []
    gen_energy_history = []
    fuel_efficiency_history = []
    fuel_usage_history = []
    scenario_history = []

    for load_kw in load_ser:

        net_power_kw = generator_power_kw - load_kw  # peak shaving

        # generator can meet demand at 100% load
        if net_power_kw == 0:
            generator_power_kw_actual = generator_power_kw
            fuel_eff = fuel_func(generator_power_kw_actual)  # kWh/litre
            generator_usage_kwh = generator_power_kw_actual * time_step_hours
            fuel_usage = (generator_usage_kwh) / fuel_eff
            fault = 0
            battery_power_kw_actual = 0
            scenario = 3 # generator only
        
        # run generator as high as possible and charge battery if possible
        elif net_power_kw > 0:
            # charge amount limited by power, net power and batter cap left
            charge_energy_kwh = min(max_charge_kwh, (net_power_kw * time_step_hours)*battery_efficiency, (battery_capacity_kwh-soc_kwh))
            
            # Generator runs at full load and battery takes extra
            if charge_energy_kwh == (net_power_kw * time_step_hours)* battery_efficiency:
                generator_power_kw_actual = generator_power_kw
                fuel_eff = fuel_func(generator_power_kw_actual)  # kWh/litre
                generator_usage_kwh = generator_power_kw_actual * time_step_hours
                fuel_usage = (generator_usage_kwh) / fuel_eff
            
            # Generator runs as high as possible and battery charges as much as it can
            else:
                gen_load_kwh = (generator_power_kw * time_step_hours) - ((net_power_kw * time_step_hours)-charge_energy_kwh)
                generator_power_kw_actual = gen_load_kwh / time_step_hours
                fuel_eff = fuel_func(generator_power_kw_actual)  # kWh/litre
                generator_usage_kwh = gen_load_kwh
                fuel_usage = (gen_load_kwh) / fuel_eff
            
            # Battery charges
            soc_kwh += charge_energy_kwh
            battery_power_kw_actual = -charge_energy_kwh / time_step_hours # negative = charging

            fault = 0
            scenario = 1 #battery charging

        elif net_power_kw < 0:
            # Generator runs at 100%
            generator_power_kw_actual = generator_power_kw
            fuel_eff = fuel_func(generator_power_kw_actual)  # kWh/litre
            generator_usage_kwh = generator_power_kw_actual * time_step_hours
            fuel_usage = (generator_usage_kwh) / fuel_eff

            
            # discharge amount limited by power, net power and soc
            discharge_energy_kwh = min(max_discharge_kwh, (abs(net_power_kw) * time_step_hours) / battery_efficiency, soc_kwh)
            
            if (
                discharge_energy_kwh == max_discharge_kwh
            ):  # battery cant supply load - clips at max discharge rate
                battery_power_kw_actual = max_discharge_kwh / time_step_hours
                fault = 1
            # battery empty
            elif discharge_energy_kwh == 0:
                battery_power_kw_actual = 0
                fault = 1
            # battery discharges required energy
            else:
                battery_power_kw_actual = (
                    discharge_energy_kwh / time_step_hours
                )  # positive = discharging
                fault = 0

            soc_kwh -= discharge_energy_kwh
            scenario = 2 #battery discharging with generator

        else:
            # no load
            battery_power_kw_actual = 0
            net_power_kw = 0
            fault = 0
            fuel_usage = 0
            fuel_eff = 0
            generator_power_kw_actual = 0
            generator_usage_kwh = 0
            scenario = 0

        soc_history.append(soc_kwh)
        battery_power_history.append(battery_power_kw_actual)
        net_power_history.append(net_power_kw)
        fault_history.append(fault)
        fuel_usage_history.append(float(fuel_usage))
        gen_power_history.append(generator_power_kw_actual)
        scenario_history.append(scenario)
        fuel_efficiency_history.append(float(fuel_eff))
        gen_energy_history.append(generator_usage_kwh)

    return pd.DataFrame(
        {
            "Load_kW": load_ser,
            "Net_power_kW": net_power_history,
            "scenario": scenario_history,
            "SOC_kWh": soc_history,
            "Battery_Power_kW": battery_power_history,
            "fault_history": fault_history,
            "gen_energy_usage": gen_energy_history,
            "fuel_eff": fuel_efficiency_history,
            "fuel_usage_history": fuel_usage_history,
            "generator_power_kW_actual": gen_power_history,
        },
        index=load_ser.index,
    )

In [ ]:
#-1 battery discharge only
# 0 no load
# 1 battery charging
# 2 battery discharging with generator

In [ ]:
#helper function
def system_rental_price(generator_power_kw, battery_capacity_kwh, battery_power_kw):
    rental_gen_cost = generator_power_kw * PERIOD_WEEKS * (20/10) #£20/10kW per week
    battery_cap_rental_cost = battery_capacity_kwh * PERIOD_WEEKS * (33/5) #£33/5kWh per week
    battery_power_rental_cost = battery_power_kw * PERIOD_WEEKS * (100/10) #£100/10kW per week
    return sum([rental_gen_cost, battery_cap_rental_cost, battery_power_rental_cost])


def fuel_cost_and_co2_faults(results_df):
    
    return  {
        "fuel_cost_£": results_df["fuel_usage_history"].sum()*PRICE_PER_LITRE,
        "co2_kg": results_df["fuel_usage_history"].sum()*CO2_LITRE,
        "no_faults": results_df['fault_history'].sum()
    }


#### Run model

In [ ]:
generator_power_kw=10
battery_capacity_kwh=37
battery_power_kw=30

results_df = simulate_battery_generator(
    load_ser = filled_load_df["Power"],
    battery_capacity_kwh = battery_capacity_kwh,
    battery_power_kw = battery_power_kw,
    generator_power_kw = generator_power_kw,
    normalised_gen_curve = gen_curve
)

rental_price = system_rental_price(generator_power_kw, battery_capacity_kwh, battery_power_kw)
fuel_cost_and_co2_faults_dict = fuel_cost_and_co2_faults(results_df)
total_price = fuel_cost_and_co2_faults_dict['fuel_cost_£'] + rental_price

print(f"The total cost = {total_price} which represents a saving of {TOTAL_GEN_COST - total_price}")
print(f"The total co2 = {fuel_cost_and_co2_faults_dict['co2_kg']} which represents a saving of {TOTAL_GEN_CO2 - fuel_cost_and_co2_faults_dict['co2_kg']}")
print(f"Total no. of faults = {fuel_cost_and_co2_faults_dict['no_faults']}")

In [ ]:
selected_columns = [
    "Load_kW",
    # "Net_power_kW",
    # "scenario",
    "SOC_kWh",
    "Battery_Power_kW",
    "fault_history",
    # "gen_energy_usage",
    "fuel_eff",
    # "fuel_usage_history",
    "generator_power_kW_actual",
]


fig = make_subplots(
    rows=len(selected_columns), cols=1,
    shared_xaxes=True,
    subplot_titles=selected_columns,
    vertical_spacing=0.05
)

for i, col in enumerate(selected_columns, start=1):
    fig.add_trace(
        go.Scatter(
            x=results_df.index, 
            y=results_df[col], 
            name=col,
            mode='lines'
        ),
        row=i, col=1
    )

fig.update_layout(
    height=100 * len(selected_columns), 
    showlegend=False
)

fig.show()

## Optimisation
Use scipy minimize and add penalty to avoid faults where the battery cannot supply the needed power

#### Minimise generator size

In [ ]:
def generator_cost_function(params):
    generator_power_kw, battery_capacity_kwh, battery_power_kw = params
    
    # Run simulation - to get faults
    results_df = simulate_battery_generator(
        load_ser=filled_load_df["Power"],
        battery_capacity_kwh=battery_capacity_kwh,
        battery_power_kw=battery_power_kw,
        generator_power_kw=generator_power_kw,  # minimize passes as array
        normalised_gen_curve = gen_curve
    )

    faults = results_df['fault_history'].sum()
    return generator_power_kw + faults * 1000  # penalize faults heavily

In [ ]:
# Bounds for battery parameters
bounds = [
    (5, 35),  # generator_power_kW
    (5, 100),# battery_capacity_kWh
    (5, 40)   # battery_power_kW
]    

# Run optimization
result = minimize(generator_cost_function, x0=[10, 40, 35], bounds=bounds)

# Extract results
min_generator_power, optimal_capacity, optimal_power = result.x

print(f"Optimal battery capacity: {optimal_capacity:.2f} kWh")
print(f"Optimal battery power: {optimal_power:.2f} kW")
print(f"Optimal generator power: {min_generator_power:.2f} kW")

In [ ]:
# gen in blocks of 10kW, battery in 10kW and 5kWh blocks
generator_power_kw=10
battery_capacity_kwh=40
battery_power_kw=30

# Calculate co2 and cost (compate to gen only)
results_df = simulate_battery_generator(
    load_ser=filled_load_df["Power"],
    battery_capacity_kwh=battery_capacity_kwh,
    battery_power_kw=battery_power_kw,
    generator_power_kw=generator_power_kw,
    normalised_gen_curve = gen_curve
)

rental_price = system_rental_price(generator_power_kw, battery_capacity_kwh, battery_power_kw)
fuel_cost_and_co2_faults_dict = fuel_cost_and_co2_faults(results_df)
total_price = fuel_cost_and_co2_faults_dict['fuel_cost_£'] + rental_price
print(fuel_cost_and_co2_faults_dict)

print(f"The total cost = {total_price} which represents a saving of {TOTAL_GEN_COST - total_price}")
print(f"The total co2 = {fuel_cost_and_co2_faults_dict['co2_kg']} which represents a saving of {TOTAL_GEN_CO2 - fuel_cost_and_co2_faults_dict['co2_kg']}")
print(f"Total no. of faults = {fuel_cost_and_co2_faults_dict['no_faults']}")

#### Optimise for cost

In [ ]:
def system_price_cost_function(params):
    generator_power_kw, battery_capacity_kwh, battery_power_kw = params
    
    # Run simulation - to get faults and fuel price
    results_dict = simulate_battery_generator(
        load_ser=filled_load_df["Power"],
        battery_capacity_kwh=battery_capacity_kwh,
        battery_power_kw=battery_power_kw,
        initial_soc=0.5,
        generator_power_kw=generator_power_kw,  # minimize passes as array
        battery_efficiency=0.85,
        normalised_gen_curve = gen_curve
    )
    
    results_dict = fuel_cost_and_co2_faults(results_dict)
    
    gen_fuel_cost = results_dict["fuel_cost_£"]
    faults = results_dict["no_faults"]
    
    rental_costs = system_rental_price(generator_power_kw, battery_capacity_kwh, battery_power_kw)
    
    return rental_costs + gen_fuel_cost + (faults * 1000)  # penalize faults heavily

In [ ]:
# Bounds for battery parameters
bounds = [
    (10, 35),  # generator_power_kW
    (5, 50),# battery_capacity_kWh
    (10, 40)   # battery_power_kW
]   

# Run optimization
result = minimize(system_price_cost_function, x0=[30, 5, 10], bounds=bounds)

# Extract results
min_generator_power, optimal_capacity, optimal_power = result.x

print(f"Optimal battery capacity: {optimal_capacity:.2f} kWh")
print(f"Optimal battery power: {optimal_power:.2f} kW")
print(f"Optimal generator power: {min_generator_power:.2f} kW")

In [ ]:
# gen in blocks of 10kW, battery in 10kW and 5kWh blocks
generator_power_kw=30
battery_capacity_kwh=5
battery_power_kw=10

# Calculate co2 and cost (compate to gen only)
results_df = simulate_battery_generator(
    load_ser=filled_load_df["Power"],
    battery_capacity_kwh=battery_capacity_kwh,
    battery_power_kw=battery_power_kw,
    generator_power_kw=generator_power_kw,
    normalised_gen_curve = gen_curve
)

rental_price = system_rental_price(generator_power_kw, battery_capacity_kwh, battery_power_kw)
fuel_cost_and_co2_faults_dict = fuel_cost_and_co2_faults(results_df)
# print(fuel_cost_and_co2_faults_dict['fuel_cost_£'], rental_price)
# print(generator_power_kw, battery_capacity_kwh, battery_power_kw)
total_price = fuel_cost_and_co2_faults_dict['fuel_cost_£'] + rental_price

print(f"The total cost = {total_price} which represents a saving of {TOTAL_GEN_COST - total_price}")
print(f"The total co2 = {fuel_cost_and_co2_faults_dict['co2_kg']} which represents a saving of {TOTAL_GEN_CO2 - fuel_cost_and_co2_faults_dict['co2_kg']}")
print(f"Total no. of faults = {fuel_cost_and_co2_faults_dict['no_faults']}")

### Optmise for co2

In [ ]:
def system_co2_cost_function(params):
    generator_power_kw, battery_capacity_kwh, battery_power_kw = params
    
    # Run simulation - to get faults and fuel usage
    results_df = simulate_battery_generator(
        load_ser=filled_load_df["Power"],
        battery_capacity_kwh=battery_capacity_kwh,
        battery_power_kw=battery_power_kw,
        generator_power_kw=generator_power_kw,  # minimize passes as array
        normalised_gen_curve = gen_curve
    )

    results_dict = fuel_cost_and_co2_faults(results_df)
    gen_fuel_co2 =results_dict["co2_kg"]
    faults = results_dict["no_faults"]
    
    return gen_fuel_co2 + (faults * 10000)  # penalize faults heavily

In [ ]:
# Bounds for battery parameters
bounds = [
    (5, 35),  # generator_power_kW
    (5, 100),# battery_capacity_kWh
    (5, 40)   # battery_power_kW
] 

# Run optimization
result = minimize(system_co2_cost_function, x0=[10, 40, 35], bounds=bounds)

# Extract results
min_generator_power, optimal_capacity, optimal_power = result.x

print(f"Optimal battery capacity: {optimal_capacity:.2f} kWh")
print(f"Optimal battery power: {optimal_power:.2f} kW")
print(f"Optimal generator power: {min_generator_power:.2f} kW")

In [ ]:
# gen in blocks of 10kW, battery in 10kW and 5kWh blocks
generator_power_kw=10
battery_capacity_kwh=35
battery_power_kw=30

# Calculate co2 and cost (compate to gen only)
results_df = simulate_battery_generator(
    load_ser=filled_load_df["Power"],
    battery_capacity_kwh=battery_capacity_kwh,
    battery_power_kw=battery_power_kw,
    generator_power_kw=generator_power_kw,
    normalised_gen_curve = gen_curve
)

rental_price = system_rental_price(generator_power_kw, battery_capacity_kwh, battery_power_kw)
fuel_cost_and_co2_faults_dict = fuel_cost_and_co2_faults(results_df)
total_price = fuel_cost_and_co2_faults_dict['fuel_cost_£'] + rental_price

print(f"The total cost = {total_price} which represents a saving of {TOTAL_GEN_COST - total_price}")
print(f"The total co2 = {fuel_cost_and_co2_faults_dict['co2_kg']} which represents a saving of {TOTAL_GEN_CO2 - fuel_cost_and_co2_faults_dict['co2_kg']}")
print(f"Total no. of faults = {fuel_cost_and_co2_faults_dict['no_faults']}")

#### Optimise for both co2 and cost
Create joint price and co2 cost function

In [ ]:
def price_and_co2_cost_function(params):
    generator_power_kw, battery_capacity_kwh, battery_power_kw = params

     # Run simulation - to get faults and fuel usage
    results_df = simulate_battery_generator(
        load_ser=filled_load_df["Power"],
        battery_capacity_kwh=battery_capacity_kwh,
        battery_power_kw=battery_power_kw,
        generator_power_kw=generator_power_kw,  # minimize passes as array
        normalised_gen_curve = gen_curve
    )

    results_dict = fuel_cost_and_co2_faults(results_df)
    gen_fuel_co2 =results_dict["co2_kg"]
    faults = results_dict["no_faults"]
    
    gen_fuel_cost = results_dict["fuel_cost_£"]
    rental_costs = system_rental_price(generator_power_kw, battery_capacity_kwh, battery_power_kw)

    #scale costs compared to gen only
    normalised_cost = (rental_costs + gen_fuel_cost)/TOTAL_GEN_COST
    normalised_co2 = gen_fuel_co2/TOTAL_GEN_CO2
    
    #scale co2 copmared to gen only
    
    return  (0.5*normalised_cost) + (0.5*normalised_co2) + (faults * 1000)  # penalize faults heavily

In [ ]:
# Bounds for battery parameters
bounds = [
    (5, 35),  # generator_power_kW
    (5, 100),# battery_capacity_kWh
    (5, 40)   # battery_power_kW
] 

# Run optimization
result = minimize(price_and_co2_cost_function, x0=[10, 40, 35], bounds=bounds)

# Extract results
min_generator_power, optimal_capacity, optimal_power = result.x

print(f"Optimal battery capacity: {optimal_capacity:.2f} kWh")
print(f"Optimal battery power: {optimal_power:.2f} kW")
print(f"Optimal generator power: {min_generator_power:.2f} kW")

In [ ]:
generator_power_kw=10
battery_capacity_kwh=40
battery_power_kw=30

# Calculate co2 and cost (compate to gen only)
results_df = simulate_battery_generator(
    load_ser=filled_load_df["Power"],
    battery_capacity_kwh=battery_capacity_kwh,
    battery_power_kw=battery_power_kw,
    generator_power_kw=generator_power_kw,
    normalised_gen_curve = gen_curve
)

rental_price = system_rental_price(generator_power_kw, battery_capacity_kwh, battery_power_kw)
fuel_cost_and_co2_faults_dict = fuel_cost_and_co2_faults(results_df)
total_price = fuel_cost_and_co2_faults_dict['fuel_cost_£'] + rental_price

print(f"The total cost = {total_price} which represents a saving of {TOTAL_GEN_COST - total_price}")
print(f"The total co2 = {fuel_cost_and_co2_faults_dict['co2_kg']} which represents a saving of {TOTAL_GEN_CO2 - fuel_cost_and_co2_faults_dict['co2_kg']}")
print(f"Total no. of faults = {fuel_cost_and_co2_faults_dict['no_faults']}")